In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

zsh:1: command not found: nvidia-smi


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

2021-09-17 10:22:06.824078: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-17 10:22:06.824103: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [3]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_norm = scaler.transform(X)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [13]:
class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # The scaling vector could be [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        # and the resulting weighted values could be
        #
        # [[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_vector = tf.cumsum(tf.constant(self.regularization_penalty, shape=(x.shape[-1],), dtype=dtype), axis=0)
        weighted_values = scaling_vector * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class Dense(tf.keras.Model):
    def __init__(self, input_units, units, activation, regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = Dense(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy


class Sequential(tf.keras.Model):
    def __init__(self, layers, activation=None):
        super().__init__()
        
        self.lrs = list()
        for layer in layers:
            self.lrs.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.lrs:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.lrs) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def remove_regularization(self):
        for l in range(len(self.lrs)):
            self.lrs[l] = self.lrs[l].copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.lrs[0].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for l in range(0, len(self.lrs) - 1):  # Every layer except of the last is regularized
            self.lrs[l].regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        for l in range(len(self.lrs) - 1):
            layer1 = self.lrs[l]
            layer2 = self.lrs[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, percentage, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.lrs) - 1):
            layer1 = self.lrs[l]
            layer2 = self.lrs[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * percentage))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)
    
    @staticmethod
    def get_param_string(weights, bias):
        param_string = ""
        weights_with_bias = tf.concat([weights, tf.reshape(bias, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(self.get_param_string(layer.W, layer.b))
    
    def evaluate(self, x, y, validation_data):
        x_val = validation_data[0]
        y_val = validation_data[1]

        y_pred = self(x)
        loss = float(tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, y_pred)))
        accuracy = float(tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y, y_pred)))
        
        y_val_pred = self(x_val)
        val_loss = float(tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_val, y_val_pred)))
        val_accuracy = float(tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_val, y_val_pred)))

        return loss, accuracy, val_loss, val_accuracy
    
    def print_epoch_statistics(self, x, y, validation_data, print_neurons):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, validation_data)
        print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total neurons: {sum(hidden_layer_sizes)}")
        if print_neurons:
            self.print_neurons()
    
    def update_history(self, x, y, validation_data, history):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, validation_data)
        history['loss'].append(loss)
        history['accuracy'].append(accuracy)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

    def fit(self, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, 
            regularization_penalty_multiplier=1., stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, verbose=True, print_neurons=False):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

        history = {
            'loss': list(),
            'accuracy': list(),
            'val_loss': list(),
            'val_accuracy': list(),
        }

        best_val_loss = np.inf
        training_stalled = False
        for epoch in range(epochs):
            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch + 1}/{epochs}")

            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before growing:")
                    self.print_epoch_statistics(x, y, validation_data, print_neurons)

                loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, validation_data)
                if val_loss >= best_val_loss * stall_coefficient:
                    if not training_stalled:
                        penalty = self.get_regularization_penalty() * regularization_penalty_multiplier
                        self.set_regularization_penalty(penalty)
                        training_stalled = True
                else:
                    best_val_loss = val_loss
                    training_stalled = False

                self.grow(percentage=growth_percentage, min_new_neurons=min_new_neurons, scaling_factor=pruning_threshold)
                if verbose:
                    print("After growing:")
                    self.print_epoch_statistics(x, y, validation_data, print_neurons)
            
            if epoch == self_scaling_epochs:
                self.remove_regularization()

            for mini_epoch in range(mini_epochs_per_epoch):
                for step, (x_batch, y_batch) in enumerate(train_dataset):
                    with tf.GradientTape() as tape:
                        y_pred = self(x_batch, training=True)
                        loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                        loss_value += sum(self.losses)

                    grads = tape.gradient(loss_value, self.trainable_variables)
                    optimizer.apply_gradients(zip(grads, self.trainable_variables))
            
            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before pruning:")
                    self.print_epoch_statistics(x, y, validation_data, print_neurons)
                self.prune(threshold=pruning_threshold)
                if verbose:
                    print("After pruning:")
                    self.print_epoch_statistics(x, y, validation_data, print_neurons)
            else:
                if verbose:
                    self.print_epoch_statistics(x, y, validation_data, print_neurons)
            
            self.update_history(x, y, validation_data, history)

        return history

# Convolutions

In [8]:
epochs = 2
self_scaling_epochs = 2
batch_size = 32
min_new_neurons = 20

In [16]:
%%time

model = Sequential([
        Dense(3072, 30, activation='selu', regularization_penalty=0.001, regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(30, 30, activation='selu', regularization_penalty=0.001, regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(30, 10, activation='softmax', regularization_penalty=0., regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/2
Before growing:
loss: 2.888000965118408 - accuracy: 0.08420000225305557 - val_loss: 2.8840060234069824 - val_accuracy: 0.08309999853372574 - penalty: 0.001
hidden layer sizes: [30, 30], total neurons: 60
After growing:
loss: 2.8880012035369873 - accuracy: 0.08420000225305557 - val_loss: 2.8840060234069824 - val_accuracy: 0.08309999853372574 - penalty: 0.001
hidden layer sizes: [50, 50], total neurons: 100
Before pruning:
loss: 1.8788080215454102 - accuracy: 0.3310999870300293 - val_loss: 1.882436752319336 - val_accuracy: 0.32659998536109924 - penalty: 0.001
hidden layer sizes: [50, 50], total neurons: 100
After pruning:
loss: 1.878679871559143 - accuracy: 0.3349800109863281 - val_loss: 1.8801465034484863 - val_accuracy: 0.33469998836517334 - penalty: 0.001
hidden layer sizes: [9, 30], total neurons: 39
##########################################################
Epoch 2/2
Before growing:
loss: 1.878679871559143 - accura

{'loss': [1.878679871559143, 1.8319804668426514],
 'accuracy': [0.3349800109863281, 0.34516000747680664],
 'val_loss': [1.8801465034484863, 1.8372929096221924],
 'val_accuracy': [0.33469998836517334, 0.34470000863075256]}

In [57]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3072, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(117, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(109, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(104, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(139, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='lecun_normal'),
])

In [58]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [59]:
%%time

model.fit(X_norm_automobiles_dogs_train, y_automobiles_dogs_train, epochs=20, validation_data=(X_norm_automobiles_dogs_test, y_automobiles_dogs_test))

Epoch 1/20
263/263 [==============================] - 1s 4ms/step - loss: 0.4151 - accuracy: 0.8404 - val_loss: 0.3217 - val_accuracy: 0.8722
Epoch 2/20
263/263 [==============================] - 1s 4ms/step - loss: 0.2258 - accuracy: 0.9095 - val_loss: 0.2861 - val_accuracy: 0.8914
Epoch 3/20
263/263 [==============================] - 1s 4ms/step - loss: 0.1857 - accuracy: 0.9270 - val_loss: 0.2878 - val_accuracy: 0.8933
Epoch 4/20
263/263 [==============================] - 1s 4ms/step - loss: 0.1265 - accuracy: 0.9476 - val_loss: 0.2857 - val_accuracy: 0.8989
Epoch 5/20
263/263 [==============================] - 1s 4ms/step - loss: 0.0938 - accuracy: 0.9640 - val_loss: 0.2879 - val_accuracy: 0.9103
Epoch 6/20
263/263 [==============================] - 1s 4ms/step - loss: 0.0858 - accuracy: 0.9680 - val_loss: 0.3245 - val_accuracy: 0.9025
Epoch 7/20
263/263 [==============================] - 1s 4ms/step - loss: 0.0887 - accuracy: 0.9683 - val_loss: 0.3448 - val_accuracy: 0.8972
Epoch 